In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Epilepsy"
cohort = "GSE123993"

# Input paths
in_trait_dir = "../../input/GEO/Epilepsy"
in_cohort_dir = "../../input/GEO/Epilepsy/GSE123993"

# Output paths
out_data_file = "../../output/preprocess/Epilepsy/GSE123993.csv"
out_gene_data_file = "../../output/preprocess/Epilepsy/gene_data/GSE123993.csv"
out_clinical_data_file = "../../output/preprocess/Epilepsy/clinical_data/GSE123993.csv"
json_path = "../../output/preprocess/Epilepsy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"No effect of calcifediol supplementation on skeletal muscle transcriptome in vitamin D deficient frail older adults."
!Series_summary	"Vitamin D deficiency is common among older adults and has been linked to muscle weakness. Vitamin D supplementation has been proposed as a strategy to improve muscle function in older adults. The aim of this study was to investigate the effect of calcifediol (25-hydroxycholecalciferol) on whole genome gene expression in skeletal muscle of vitamin D deficient frail older adults. A double-blind placebo controlled trial was conducted in vitamin D deficient frail older adults (aged above 65), characterized by blood 25-hydroxycholecalciferol concentrations between 20 and 50 nmol/L. Subjects were randomized across the placebo group (n=12) and the calcifediol group (n=10, 10 µg per day). Muscle biopsies were obtained before and after six months of calcifediol or placebo supplementation and subjected to whole genome gene e

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Assess gene expression data availability
is_gene_available = True  # Based on background info, this dataset contains microarray gene expression data from muscle biopsies

# 2. Variable availability and data type conversion
# 2.1 & 2.2 Trait (Intervention group)
trait_row = 3  # Key for intervention group
def convert_trait(value):
    if ":" not in value:
        return None
    value = value.split(":", 1)[1].strip()
    if "25-hydroxycholecalciferol" in value or "25(OH)D3" in value:
        return 1
    elif "Placebo" in value:
        return 0
    return None

# Age data is not available in the sample characteristics
age_row = None
def convert_age(value):
    return None  # Not used but defined for completeness

# Gender data
gender_row = 1  # Key for Sex information
def convert_gender(value):
    if ":" not in value:
        return None
    value = value.split(":", 1)[1].strip()
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    return None

# 3. Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical feature extraction
if trait_row is not None:
    try:
        # Create a DataFrame to store the clinical features
        sample_ids = []
        gender_values = []
        trait_values = []
        
        # Sample characteristics dictionary from previous step
        sample_chars = {0: ['tissue: muscle'], 
                    1: ['Sex: Male', 'Sex: Female'], 
                    2: ['subject id: 3087', 'subject id: 3088', 'subject id: 3090', 'subject id: 3106', 
                        'subject id: 3178', 'subject id: 3241', 'subject id: 3258', 'subject id: 3279', 
                        'subject id: 3283', 'subject id: 3295', 'subject id: 3322', 'subject id: 3341', 
                        'subject id: 3360', 'subject id: 3361', 'subject id: 3375', 'subject id: 3410', 
                        'subject id: 3430', 'subject id: 3498', 'subject id: 3516', 'subject id: 3614', 
                        'subject id: 3695', 'subject id: 3731'], 
                    3: ['intervention group: 25-hydroxycholecalciferol (25(OH)D3)', 'intervention group: Placebo'], 
                    4: ['time of sampling: before intervention (baseline)', 'time of sampling: after intervention']}
        
        # Manually create clinical DataFrame with the necessary columns
        clinical_df = pd.DataFrame()
        
        # Add trait column
        trait_values = []
        for value in sample_chars[trait_row]:
            trait_values.append(convert_trait(value))
        clinical_df[trait] = trait_values
        
        # Add gender column if available
        if gender_row is not None:
            gender_values = []
            for value in sample_chars[gender_row]:
                gender_values.append(convert_gender(value))
            clinical_df['Gender'] = gender_values
        
        # Preview extracted data
        print("Preview of clinical features:")
        print(preview_df(clinical_df))
        
        # Save clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
        
    except Exception as e:
        print(f"Error during clinical feature extraction: {e}")


Preview of clinical features:
{'Epilepsy': [1, 0], 'Gender': [1, 0]}
Clinical data saved to ../../output/preprocess/Epilepsy/clinical_data/GSE123993.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Epilepsy/GSE123993/GSE123993_family.soft.gz
Matrix file: ../../input/GEO/Epilepsy/GSE123993/GSE123993_series_matrix.txt.gz
Found the matrix table marker in the file.


Gene data shape: (53617, 44)
First 20 gene/probe identifiers:
['16650001', '16650003', '16650005', '16650007', '16650009', '16650011', '16650013', '16650015', '16650017', '16650019', '16650021', '16650023', '16650025', '16650027', '16650029', '16650031', '16650033', '16650035', '16650037', '16650041']


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers in the data
# The identifiers appear to be probe IDs ('16650001', '16650003', etc.) rather than
# standard human gene symbols (like BRCA1, TP53, etc.)
# These numeric identifiers are likely microarray probe IDs that need to be mapped
# to actual gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Based on the preview, 'ID' appears to be the probe ID and 'Symbol' contains gene names
# Display more samples from the Symbol column to better understand the format
print("\nSample of Symbol column (first 5 rows):")
if 'Symbol' in gene_annotation.columns:
    for i in range(min(5, len(gene_annotation))):
        print(f"Row {i}: {gene_annotation['Symbol'].iloc[i]}")

# Check the quality and completeness of the mapping
if 'Symbol' in gene_annotation.columns:
    non_null_symbols = gene_annotation['Symbol'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nSymbol column completeness: {non_null_symbols}/{total_rows} rows ({non_null_symbols/total_rows:.2%})")



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'GO_biological_process', 'GO_cellular_component', 'GO_molecular_function', 'pathway', 'protein_domains', 'crosshyb_type', 'category', 'GB_ACC', 'SPOT_ID']
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DE

### Step 6: Gene Identifier Mapping

In [7]:
# Extract the gene expression data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_expression = get_genetic_data(matrix_file)

# Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# Examine the gene_annotation and gene_expression to identify mapping columns
print(f"Gene expression dataset shape: {gene_expression.shape}")

# Look at the gene annotation columns
print(f"Gene annotation columns: {gene_annotation.columns.tolist()}")

# Based on the previews, the ID column contains the probe IDs (like '16650001')
# The 'gene_assignment' column appears to contain gene information
# First, let's confirm a few probe IDs to ensure proper mapping
print("Sample probe IDs from gene expression data:")
print(gene_expression.index[:5].tolist())

print("Sample probe IDs from gene annotation data:")
print(gene_annotation['ID'].head(5).tolist())

# Define the column names for mapping
probe_col = 'ID'  # The gene identifiers in the expression data
gene_col = 'gene_assignment'  # Contains gene symbols

# Create mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_expression, gene_mapping)

# Check the resulting gene data
print(f"Gene data shape: {gene_data.shape}")
print("First 5 gene symbols:")
print(gene_data.index[:5].tolist())

# Save gene data to output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene expression dataset shape: (53617, 44)
Gene annotation columns: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'GO_biological_process', 'GO_cellular_component', 'GO_molecular_function', 'pathway', 'protein_domains', 'crosshyb_type', 'category', 'GB_ACC', 'SPOT_ID']
Sample probe IDs from gene expression data:
['16650001', '16650003', '16650005', '16650007', '16650009']
Sample probe IDs from gene annotation data:
['16657436', '16657440', '16657445', '16657447', '16657450']


Gene data shape: (81076, 44)
First 5 gene symbols:
['A-', 'A-2', 'A-52', 'A-E', 'A-I']


Gene expression data saved to ../../output/preprocess/Epilepsy/gene_data/GSE123993.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data - respecting the analysis from Step 2
# From Step 2, we determined:
# trait_row = None  # No Epilepsy data available
# age_row = None
# gender_row = None
is_trait_available = trait_row is not None

# Skip clinical feature extraction when trait_row is None
if is_trait_available:
    try:
        # Load the clinical data from file
        soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            gender_row=gender_row,
            convert_gender=convert_gender,
            age_row=age_row,
            convert_age=convert_age
        )
        
        print(f"Extracted clinical data shape: {clinical_features.shape}")
        print("Preview of clinical data (first 5 samples):")
        print(clinical_features.iloc[:, :5])
        
        # Save the properly extracted clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical data: {e}")
        is_trait_available = False
else:
    print("No trait data (Epilepsy) available in this dataset based on previous analysis.")

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # Create an empty DataFrame for metadata purposes
    empty_df = pd.DataFrame()
    
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=empty_df,  # Empty dataframe for metadata
        note="Missing essential data components for linking: dataset contains gene expression data from GBM cell cultures, but no epilepsy phenotype information."
    )

Gene data shape before normalization: (81076, 44)
Gene data shape after normalization: (23274, 44)


Normalized gene data saved to ../../output/preprocess/Epilepsy/gene_data/GSE123993.csv
Extracted clinical data shape: (2, 44)
Preview of clinical data (first 5 samples):
          GSM3518336  GSM3518337  GSM3518338  GSM3518339  GSM3518340
Epilepsy         1.0         1.0         1.0         1.0         1.0
Gender           1.0         1.0         0.0         0.0         0.0
Clinical data saved to ../../output/preprocess/Epilepsy/clinical_data/GSE123993.csv
Gene data columns (first 5): ['GSM3518336', 'GSM3518337', 'GSM3518338', 'GSM3518339', 'GSM3518340']
Clinical data columns (first 5): ['GSM3518336', 'GSM3518337', 'GSM3518338', 'GSM3518339', 'GSM3518340']
Found 44 common samples between gene and clinical data
Initial linked data shape: (44, 23276)
Preview of linked data (first 5 rows, first 5 columns):
            Epilepsy  Gender      A1BG  A1BG-AS1      A1CF
GSM3518336       1.0     1.0  2.312032  0.931107  0.517549
GSM3518337       1.0     1.0  2.310189  0.924850  0.525211
GSM35183

Linked data shape after handling missing values: (44, 23276)
For the feature 'Epilepsy', the least common label is '1.0' with 20 occurrences. This represents 45.45% of the dataset.
The distribution of the feature 'Epilepsy' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 20 occurrences. This represents 45.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Epilepsy/cohort_info.json


Linked data saved to ../../output/preprocess/Epilepsy/GSE123993.csv
